В этом ноутбуке обучим модель RoBERTa распознавать спам в email

In [1]:
model_name = 'roberta'
train_dataset_name = 'spam emails'

In [2]:
MAX_LEN = 128 

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

In [5]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [7]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_emails.csv', encoding = "ISO-8859-1")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [8]:
df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
df.columns = ['DATA_COLUMN', 'IS_SPAM']

In [9]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...
5166,Subject: put the 10 on the ft\r\nthe transport...,0
5167,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,Subject: industrial worksheets for august 2000...,0


In [10]:
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))

In [11]:
parameter_const = 1.8 # подобрали
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x[:round(MAX_LEN * parameter_const)])

In [12]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291 thi...,0
1,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs this deal is to b...,0
...,...,...
5166,Subject: put the 10 on the ft the transport vo...,0
5167,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,Subject: calpine daily gas nomination > > juli...,0
5169,Subject: industrial worksheets for august 2000...,0


In [13]:
df['IS_SPAM'].value_counts()

0    3672
1    1499
Name: IS_SPAM, dtype: int64

In [14]:
df_positive = df[df['IS_SPAM']==1]

In [15]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [16]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [17]:
df_negative_test.shape

(1836, 2)

In [18]:
df_positive_test.shape

(749, 2)

In [19]:
df_positive_test

,DATA_COLUMN,IS_SPAM
2699,Subject: your mother knows you watch girls moa...,1
2702,Subject: 65 % off for all new software . pyrrh...,1
2703,Subject: [ wrenches ] 68 % off dreamweaver mx ...,1
2706,Subject: he reached around and fingered me whi...,1
2709,"Subject: best meds around , paliourg haven ' t...",1
...,...,...
5159,Subject: pictures streamlined denizen ajar cha...,1
5161,Subject: penny stocks are about timing nomad i...,1
5162,Subject: anomaly boys from 3881 uosda apaprove...,1
5164,Subject: slutty milf wants to meet you take th...,1


In [20]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [21]:
df_balanced_test.sample(10)

,DATA_COLUMN,IS_SPAM
4930,Subject: - complimentary computer - sony vaio ...,1
3519,Subject: fw : payment jo hillier - smith owner...,0
3787,Subject: new municipal toxic emmissions report...,1
3942,Subject: revisions on tennessee gas p / l effe...,0
3474,Subject: paliourg udtih 7 wcwknoanopkt good mo...,1
5128,Subject: hplc / tglo deals darren : i have a r...,0
3072,Subject: meter 986315 for dec . and jan . prod...,0
4155,Subject: sarco lateral and crow o ' connor met...,0
4117,Subject: meter 1558 april 2001 cerrito lastien...,0
4560,Subject: my site links to your site now hello ...,1


In [22]:
df_balanced_test['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [23]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [24]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [25]:
df_balanced_train['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [26]:
df_balanced_train.sample(10)

,DATA_COLUMN,IS_SPAM
1136,Subject: re : deal extension for 11 / 21 / 200...,0
1564,Subject: your pharmacy nx want a cheap pain ki...,1
936,Subject: pops it team temporarily moving to al...,0
2186,Subject: revision - forest oil ' s november 20...,0
396,Subject: sitara updates this is a note to upda...,0
1160,Subject: shell meter 1581 this meter is coming...,0
2067,Subject: message subject hey i ' am julie ^ _ ...,1
1852,Subject: re : sitara training i appreciate it ...,0
1202,"Subject: logistics training daren , thanks for...",0
1247,Subject: meter 1459 - 8 / 00 daren - can you g...,0


In [27]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze().values
y_train = df_balanced_train['IS_SPAM'].squeeze().values

In [28]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze().values
y_test = df_balanced_test['IS_SPAM'].squeeze().values

Работаем с моделью

In [29]:
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [30]:
max_len = MAX_LEN
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.BinaryCrossentropy()

input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
output = roberta_model.roberta([input_ids,attention_masks])
output = output[1]
output = tf.keras.layers.Dense(1, activation='sigmoid')(output)
model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer=opt, loss=loss, metrics=METRICS)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [31]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")

In [32]:
# код ниже нужен для проверки
token_lens = []

for txt in X_train:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 120


In [33]:
for txt in X_test:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 120


In [34]:
def tokenize_roberta(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [35]:
train_input_ids, train_attention_masks = tokenize_roberta(X_train, MAX_LEN)
test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)

In [36]:
history = model.fit([train_input_ids,train_attention_masks], y_train, epochs=5, batch_size=30)

Epoch 1/5
87/87 [==============================] - 122s 797ms/step - loss: 0.2719 - accuracy: 0.8565 - precision: 0.7917 - recall: 0.6849
Epoch 2/5
87/87 [==============================] - 64s 740ms/step - loss: 0.0565 - accuracy: 0.9818 - precision: 0.9705 - recall: 0.9666
Epoch 3/5
87/87 [==============================] - 64s 734ms/step - loss: 0.0257 - accuracy: 0.9903 - precision: 0.9788 - recall: 0.9880
Epoch 4/5
87/87 [==============================] - 64s 738ms/step - loss: 0.0124 - accuracy: 0.9942 - precision: 0.9880 - recall: 0.9920
Epoch 5/5
87/87 [==============================] - 64s 731ms/step - loss: 0.0099 - accuracy: 0.9965 - precision: 0.9920 - recall: 0.9960


In [37]:
result_roberta = model.predict([test_input_ids,test_attention_masks])

81/81 [==============================] - 25s 269ms/step


In [38]:
y_predicted = np.where(result_roberta > 0.5, 1, 0)
y_predicted


array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [39]:
y_test

array([0, 0, 0, ..., 1, 1, 1])

In [40]:
accuracy_score(y_test, y_predicted)

0.9771760154738878

In [41]:
precision_score(y_test, y_predicted)


0.9831932773109243

In [42]:
recall_score(y_test, y_predicted)

0.9372496662216289

In [43]:
f1_score(y_test, y_predicted)

0.959671907040328

In [44]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [45]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [46]:
df_results_on_test

,accuracy,precision,recall,f1_score
roberta trained on spam emails and tested on spam emails dataset,0.977176,0.983193,0.93725,0.959672


Сохраним обученную модель

In [47]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/roberta_trained_on_spam_emails_20_january'

In [48]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/roberta_trained_on_spam_emails_20_january


In [49]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [50]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [51]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [52]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [53]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [54]:
df_with_train_quality

,accuracy,precision,recall,f1_score
roberta trained on spam emails quality on train dataset,0.996518,0.992021,0.995995,0.994004


In [55]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [56]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [57]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')